In [8]:
import scipy as sc
from scipy import stats, optimize, interpolate
import numpy
import functions
import autograd as ad
from autograd.variable import Variable
from autograd.optimize import Adam
from autograd.optimize import GD

In [2]:
guess = numpy.zeros((1,11))

In [3]:
result = sc.optimize.minimize(functions.error,guess, method = 'CG')
planar = result.jac
W,w,wo =functions.convert(planar)
print("Error: ",result.fun)
print("W: ", W)
print("w: ", w)
print("wo: ", wo)

Error:  4.693377633449785e-06
W:  [-8.60955640e-06 -3.78984851e-07 -3.78984851e-07]
w:  (array([ 9.20863386e-12,  1.38129508e-11, -9.15179044e-12]), array([ 9.20863386e-12,  1.38129508e-11, -9.15179044e-12]))
wo:  [0. 0.]


In [4]:
print(sc.optimize.fmin_cg(functions.error,guess))

Optimization terminated successfully.
         Current function value: 0.000005
         Iterations: 12
         Function evaluations: 756
         Gradient evaluations: 63
[ 6.15316712  7.1193765   7.1193765  -2.76147322  0.53919878  2.34582673
 -2.76147322  0.53919878  2.34582673  0.06293751  0.06293751]


In [10]:
result = Adam(functions.error,guess)
result.solve()

In [7]:
result = GD(functions.error,guess).solve()

IndexError: index 1 is out of bounds for axis 0 with size 1